In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
url = "https://kienthuckhoahoc.org/"

In [22]:
topics = list()

res = requests.get(url)
soup = bs(res.content)
title_div = soup.find('div', class_ = 'navbox clearfix')
titles = title_div.find_all('li')

for element in titles:
  title = element.find('a')
  topics.append(title['href'])

In [24]:
print(topics)

['/khao-co', '/video', '/anh-dep', '/dai-duong-hoc', '/su-kien-kh', '/9999-bi-an', '/cau-chuyen-khoa-hoc', '/cong-trinh-khoa-hoc', '/thu-gian', '/di-san', '/lich-su', '/kh-va-ban-doc', '/suc-khoe-y-hoc', '/suc-khoe-y-hoc/suc-khoe', '/cong-nghe/cong-nghe-moi', '/moi-truong', '/moi-truong/tham-hoa', '/sinh-vat/con-trung-vi-khuan', '/sinh-vat/thuc-vat', '/ung-dung-kh', '/moi-truong/thoi-tiet', '/moi-truong/giai-phap', '/moi-truong/thien-nhien', '/suc-khoe-y-hoc/dich-cum', '/cong-nghe/kh-may-tinh', '/sinh-vat/cong-nghe-sinh-hoc', '/suc-khoe-y-hoc/ung-thu', '/moi-truong/tin-bao', '/cong-nghe/phat-minh-kh', '/suc-khoe-y-hoc/thuc-pham', '/di-san/van-hoa', '/lich-su/ngay-nam-xua', '/di-san/thien-nhien', '/cong-nghe/phan-mem-hay', '/di-san/phi-vat-the', '/di-san/hon-hop', '/di-san/tu-lieu']


In [4]:
def get_article(url):
    article_name = list()
    res = requests.get(url)
    soup = bs(res.content)
    articles = soup.find('div', class_ = "listview")
    if articles:
        li_elements = articles.find_all('li')
        for article in li_elements:
            if article.find('a'):
                article_name.append(article.find('a')['href'])
    return article_name        

In [5]:
topic_article = dict()   #key: topic - value: list of articles's link

for topic in topics:
    topic_article[topic] = list()
    for page in range(2, 150):
        link = f"{url}{topic}/page{page}"
        article_name = get_article(link)
        topic_article[topic].extend(article_name)
    

In [7]:
len(topic_article['/suc-khoe-y-hoc/suc-khoe'])

2220

In [ ]:
def crawl(link):
    paragraph = ""
    res = requests.get(link)
    soup = bs(res.content)
    contents = soup.find('div', class_ = "content-detail")
    if contents:
        contents = contents.find_all('p')
        for content in contents:
            if "Nguồn bài viết: " in str(content):
                break
            if 'div class=' not in str(content):
                paragraph += str(content)
    return paragraph

import os
def save(path, paragraph, index):
    if not os.path.exists(path):
        os.makedirs(path)
    path += f"/{index}.txt"
    with open(path, "w") as file:
        file.write(paragraph)

for topic in topic_article.keys():
    for index, article in enumerate(topic_article[topic]):
        link = f"{url}{article}"
        paragraph = crawl(link)
        path = f"data/{topic}"
        save(path, paragraph, index)
